<a href="https://colab.research.google.com/github/rohitshinde4680/marker_Cluster/blob/main/riverTimelapse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import geemap
import ee
ee.Authenticate()

True

In [20]:
ee.Initialize(project= "ee-rohitshinde090103")

In [21]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [22]:
India = ee.FeatureCollection("projects/rohitshinde4680/assets/India_state")
m.addLayer(India, {}, "India")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [23]:
Godavari = ee.FeatureCollection("projects/ee-rohitshinde913/assets/godavari")
m.addLayer(Godavari, {}, "Godavari")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [24]:
buffer_distance = 50000
Godavari_buffer = Godavari.map(lambda f: f.buffer(buffer_distance))
m.addLayer(Godavari_buffer, {}, "Godavari_buffer")
m


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [25]:
NDWI = ee.ImageCollection("LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI").select('NDWI')

In [26]:
NDWI_Godavari = NDWI.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = NDWI_Godavari.filterDate('2022-01-01', '2023-12-31');

In [27]:
filter = ee.Filter.equals(leftField='doy', rightField ='doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDWI_Godavari, filter))

In [28]:
def add_date_from(img):
  doy = ee.Number(img.get('doy'))

In [36]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [29]:
Vis_para = {
  'min': 0.0,
  'max': 0.1,
  'palette': [
    '0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'
  ],
};

In [37]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDWI_mean'],**Vis_para).clip(Godavari_buffer))

In [39]:
gifParams = {
  'region': Godavari_buffer.geometry(),
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 80,
  'format':'gif',
  'addDate': 'yyyy-mm-dd'
}

In [40]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-rohitshinde090103/videoThumbnails/7612116fe59ceca92368adf02da3a7c1-1f82ef15671e62d71ab6cd86d6059794:getPixels
